In [68]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from pprint import pprint
import json
import random
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tqdm import trange

In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [73]:
TRAIN_X_PATH = 'preprocessed_selected_features/train/aux.csv'
TRAIN_Y_PATH = 'preprocessed_selected_features/train/labels.csv'

In [74]:
train_X_df = pd.read_csv(TRAIN_X_PATH)
train_y_df = pd.read_csv(TRAIN_Y_PATH)
train_X_df.set_index('participant_id', inplace=True)
train_y_df.set_index('participant_id', inplace=True)
train_y_df = train_y_df.reindex(train_X_df.index)

In [75]:
train_X_df.head()

,Barratt_Barratt_P1_Occ_20.0,Basic_Demos_Enroll_Year_2019,Barratt_Barratt_P1_Occ_35.0,Barratt_Barratt_P2_Occ_10.0,APQ_P_APQ_P_PM,Barratt_Barratt_P1_Edu_12.0,Barratt_Barratt_P2_Edu_21.0,Barratt_Barratt_P2_Edu_9.0,Barratt_Barratt_P1_Edu_15.0,Barratt_Barratt_P1_Edu_18.0,...,SDQ_SDQ_Prosocial,APQ_P_APQ_P_INV,Barratt_Barratt_P2_Edu_18.0,Barratt_Barratt_P1_Occ_5.0,APQ_P_APQ_P_OPD,Basic_Demos_Study_Site_3,Barratt_Barratt_P2_Edu_12.0,SDQ_SDQ_Peer_Problems,Barratt_Barratt_P2_Occ_45.0,Barratt_Barratt_P2_Occ_20.0
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,0,1,0,0,0.46875,0,1,0,0,0,...,0.9,0.800000,0,0,0.333333,0,0,0.222222,1,0
00fV0OyyoLfw,0,0,0,0,0.71875,0,1,0,0,0,...,0.8,0.500000,0,0,0.857143,0,0,0.333333,1,0
04X1eiS79T4B,0,0,0,0,0.65625,0,1,0,0,0,...,0.7,0.566667,0,0,0.523810,0,0,0.777778,1,0
05ocQutkURd6,0,0,0,0,0.46875,0,0,0,0,1,...,0.6,0.733333,1,0,0.380952,0,0,0.222222,0,0
06YUNBA9ZRLq,0,0,0,0,0.21875,1,1,0,0,0,...,0.4,0.500000,0,0,0.714286,0,0,0.111111,1,0


In [76]:
train_y_df.head()

,ADHD_Outcome,Sex_F
participant_id,,
00aIpNTbG5uh,1,0
00fV0OyyoLfw,1,0
04X1eiS79T4B,0,1
05ocQutkURd6,0,1
06YUNBA9ZRLq,1,0


In [77]:
class Model(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout=0.5, output_dim=4):
        super(Model, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in layer_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [78]:
temp_model = Model(100, [64, 32], dropout=0.3, output_dim=4).to(device)
summary(temp_model, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]           6,464
              ReLU-2                   [-1, 64]               0
           Dropout-3                   [-1, 64]               0
            Linear-4                   [-1, 32]           2,080
              ReLU-5                   [-1, 32]               0
           Dropout-6                   [-1, 32]               0
            Linear-7                    [-1, 4]             132
Total params: 8,676
Trainable params: 8,676
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [79]:
X = np.array(train_X_df.values, dtype=np.float32)

In [80]:
X.shape

(1213, 59)

In [81]:
y_two_vars = train_y_df.values
y = np.array(y_two_vars[:, 0] * 2 + y_two_vars[:, 1], dtype=np.uint8)

In [82]:
y[:3], y_two_vars[:3]

(array([2, 2, 1], dtype=uint8),
 array([[1, 0],
        [1, 0],
        [0, 1]]))

In [83]:
layer_dims_list = [
    [64, 32, 16],
    [128, 64, 32],
    [128, 64, 32, 16],
    [256, 128, 64, 32],
    [256, 128, 64, 32, 16],
    [512, 256, 128, 64],
    [512, 256, 128, 64, 32],
    [512, 256, 128, 64, 32, 16],
]
dropouts = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

In [84]:
criterion = nn.CrossEntropyLoss()

In [85]:
seed = 42  # Choose any fixed number
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA

In [86]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [87]:
num_epochs = 200

results = {"-".join(map(str, layer_dims)): {} for layer_dims in layer_dims_list}
epoch_history = {"-".join(map(str, layer_dims)): {} for layer_dims in layer_dims_list}
for layer_dims in layer_dims_list:
    for dropout in dropouts:
        print("complexity:", layer_dims, "dropout rate:", dropout)
        f1_scores = []
        best_epochs = []

        for fold, (train_index, test_index) in enumerate(kf.split(X)):
            model = Model(
                input_dim=X.shape[1], layer_dims=layer_dims, dropout=dropout, output_dim=4
            ).to(device)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            optimizer = optim.Adam(model.parameters(), lr=0.001)
            best_test_loss = float("inf")
            best_f1 = 0.0
            best_epoch = 0

            for epoch in range(num_epochs):
                model.train()
                optimizer.zero_grad()
                outputs = model(torch.tensor(X_train).to(device))
                loss = criterion(outputs, torch.tensor(y_train).to(device))
                loss.backward()
                optimizer.step()

                model.eval()
                with torch.no_grad():
                    test_outputs = model(torch.tensor(X_test).to(device))
                    test_loss = criterion(test_outputs, torch.tensor(y_test).to(device)).item()
                    predicted = torch.argmax(test_outputs.data, 1).cpu()
                    f1 = f1_score(y_test, predicted, average="macro")

                if test_loss < best_test_loss:
                    best_test_loss = test_loss
                    best_f1 = f1
                    best_epoch = epoch


            f1_scores.append(float(best_f1))
            best_epochs.append(best_epoch)

        results["-".join(map(str, layer_dims))][dropout] = f1_scores
        epoch_history["-".join(map(str, layer_dims))][dropout] = best_epochs

complexity: [64, 32, 16] dropout rate: 0.0
complexity: [64, 32, 16] dropout rate: 0.1
complexity: [64, 32, 16] dropout rate: 0.2
complexity: [64, 32, 16] dropout rate: 0.3
complexity: [64, 32, 16] dropout rate: 0.4
complexity: [64, 32, 16] dropout rate: 0.5
complexity: [128, 64, 32] dropout rate: 0.0
complexity: [128, 64, 32] dropout rate: 0.1
complexity: [128, 64, 32] dropout rate: 0.2
complexity: [128, 64, 32] dropout rate: 0.3
complexity: [128, 64, 32] dropout rate: 0.4
complexity: [128, 64, 32] dropout rate: 0.5
complexity: [128, 64, 32, 16] dropout rate: 0.0
complexity: [128, 64, 32, 16] dropout rate: 0.1
complexity: [128, 64, 32, 16] dropout rate: 0.2
complexity: [128, 64, 32, 16] dropout rate: 0.3
complexity: [128, 64, 32, 16] dropout rate: 0.4
complexity: [128, 64, 32, 16] dropout rate: 0.5
complexity: [256, 128, 64, 32] dropout rate: 0.0
complexity: [256, 128, 64, 32] dropout rate: 0.1
complexity: [256, 128, 64, 32] dropout rate: 0.2
complexity: [256, 128, 64, 32] dropout rate

In [88]:
results_json = json.dumps(results, indent=4)
print(results_json)

{
    "64-32-16": {
        "0.0": [
            0.3588465586325827,
            0.2851851851851852,
            0.3319048408488064,
            0.30096892749749415,
            0.31938106395631816
        ],
        "0.1": [
            0.4199494093679866,
            0.2503018556176919,
            0.37201143451143454,
            0.26815714654949596,
            0.2840175150519978
        ],
        "0.2": [
            0.3101174372848008,
            0.25992414664981034,
            0.2971311475409836,
            0.3041556867100783,
            0.305138720315814
        ],
        "0.3": [
            0.31288130474176984,
            0.2676978722488281,
            0.2987704918032787,
            0.2595907928388747,
            0.3605237622773847
        ],
        "0.4": [
            0.30559268485327173,
            0.2855846042120552,
            0.3227155727155727,
            0.2879772950444328,
            0.28279569892473116
        ],
        "0.5": [
            0.3134069

In [89]:
full_results = {}
summary_results = {}
final_epoch_history = {}

for layer_dims in results.keys():
    for dropout in results[layer_dims].keys():
        full_results[layer_dims+'-'+str(dropout)] = results[layer_dims][dropout]
        summary_results[layer_dims+'-'+str(dropout)] = float(np.mean(results[layer_dims][dropout]))
        final_epoch_history[layer_dims+'-'+str(dropout)] = epoch_history[layer_dims][dropout]

summary_results = dict(sorted(summary_results.items(), key=lambda item: item[1], reverse=True))
keys = list(summary_results.keys())
final_epoch_history = dict(sorted(final_epoch_history.items(), key=lambda item: keys.index(item[0]), reverse=False))


with open("full_results.json", "w") as f:
    json.dump(results, f, indent=4)


with open("summary_results.json", "w") as f:
    json.dump(summary_results, f, indent=4)


with open("epoch_history.json", "w") as f:
    json.dump(final_epoch_history, f, indent=4)

In [91]:
best_layer_dims, best_dropout, n_epochs = [128, 64, 32], 0, 100

In [92]:
model = Model(
    input_dim=X.shape[1], layer_dims=best_layer_dims, dropout=best_dropout, output_dim=4
)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in trange(n_epochs):
    optimizer.zero_grad()
    outputs = model(torch.tensor(X))
    loss = criterion(outputs, torch.tensor(y))
    loss.backward()
    optimizer.step()

100%|██████████| 100/100 [00:00<00:00, 275.90it/s]


In [93]:
torch.save(model.state_dict(), "128-64-32-0.0.pth")

In [94]:
model.eval()
with torch.no_grad():
    yhat = model(torch.tensor(X))
predicted = torch.argmax(yhat, 1)
f1 = f1_score(y, predicted, average="macro")
print(f1)

0.40622220307423756
